## 「風しん抗体検査・風しん第５期定期接種受託医療機関」の可視化

「[厚生労働省　風しんの追加的対策について](https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/kenkou_iryou/kenkou/kekkaku-kansenshou/rubella/index_00001.html)」で公開されている「風しん抗体検査・風しん第５期定期接種受託医療機関」を地図上に可視化する．


PDFファイルから表データを取り出し，所在地の住所をアドレス・マッチングにより経緯度座標を割り出し，地図上にその位置を表示する．

## 利用したサービスやライブラリ

### 東京大学空間情報科学研究センター「CSVアドレスマッチングサービス」

http://newspat.csis.u-tokyo.ac.jp/geocode/modules/addmatch/index.php?content_id=1


### tabula

[tabula](https://tabula.technology/)は，PDFから表データを抽出するJavaアプリケーションです（Javaの実行環境が必要です）

### tabula-py

[tabula-py](https://pypi.org/project/tabula-py/) は，Pythonから tabula を利用して PDFの表データを pandas の DataFrame として取り出すことが可能なライブラリです

### kepler-gl

[kepler.gl](https://kepler.gl) は uber のオープンソース・ソフトウェアで，ブラウザ上で地理空間情報を簡単に可視化することができます．WebGLを使っており大量のデータを高速に描画することが可能です．

juypter notebook内ではウィジェットを使って表示することが可能です．

In [ ]:
import tabula
import pandas as pd
import requests
from io import StringIO

In [ ]:
# PDFファイルの URLを指定し，tabulaを使って表データを読み込み DataFrame にする
pdf_url = "https://www.mhlw.go.jp/content/000516970.pdf"

df = tabula.read_pdf(pdf_url, 
                     pages='all', # or 読み込むページ番号をリストで指定 ex. [1,2,3]
                     lattice=True, 
                     pandas_options={'header':[0,1]}
                    )

# カラム名を変更する
column_names = ['No', '実施機関名', '郵便番号', '所在地', '電話番号', '受託範囲：風しんの抗体検査', '受託範囲：風しんの第5期の定期接種']
df.columns = column_names

In [ ]:
# 不要なカラムを削除
df.drop('No', axis='columns', inplace=True)

In [ ]:
# 文字列のクレンジング

# 混在している◯と〇や，×を 1/0 に置換する
for col in ['受託範囲：風しんの抗体検査', '受託範囲：風しんの第5期の定期接種']:
    df[col] = df[col].fillna(0)
    df[col] = df[col].replace('×', 0)
    df[col] = df[col].replace('〇', 1).replace('○', 1)
    
# 値の中の改行除去
df['実施機関名'] = df['実施機関名'].str.replace('\r','')
df['所在地'] = df['所在地'].str.replace('\r','')

# 所在地の文字列からマンション名などを分離して住所のみ取り出す
df['所在地'] = df['所在地'].str.split(' ', expand=True)[0]

## アドレスマッチング・サービスで経緯度を割り出す

In [ ]:
# DataFrameから所在地を文字列バッファにCSVデータにして出力する（エンコードは Shift_JISにしておく）
buffer = StringIO('csv-data')
df[['所在地']].to_csv(buffer, encoding='shift_jis', index=False, header=None)

In [ ]:
# アドレスマッチング・サービスの設定
url_adress_matching = 'http://newspat.csis.u-tokyo.ac.jp/geocode-cgi/geocode.cgi'

params = {
    'action':'input',
    'spat_host':'newspat.csis.u-tokyo.ac.jp',
    'spat_port':8801, # 北海道，街区レベル（経緯度・世界測地系）
    'ncolumn': [1],   # カラム番号（先頭＝1）
    'input_kanji_code':'auto',
    'output_kanji_code':'auto',
    'exact_level':0
}

buffer.seek(0)

files = {'file': ("address.csv", buffer, 'text/csv')}

# アドレスマッチング・サービスにPOST
res = requests.post(url_adress_matching, files=files, data=params)
res.headers

In [ ]:
# レスポンスの確認
res.ok

In [ ]:
# アドレスマッチングの結果のCSVデータを DataFrame にして，元の DataFrameに joinする
df_geodata = pd.read_csv( StringIO(res.text) )
df_result = df.join(df_geodata[['LocName', 'fX', 'fY', 'iConf', 'iLvl']])

In [ ]:
# 経緯度のカラムをリネームする
df_result.rename({'fX':'Longitude', 'fY':'Latitude'}, axis='columns', inplace=True)

In [ ]:
# CSVファイルに出力する
df_result.to_csv("rubella_antibody_test.csv", index=False)

## kepler.glで可視化する

In [ ]:
# keplerの表示設定
config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "rubella",
          "type": "point",
          "config": {
            "dataId": "invyer8il",
            "label": "Point",
            "color": [
              125,
              194,
              64
            ],
            "columns": {
              "lat": "Latitude",
              "lng": "Longitude",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                50
              ],
              "filled": True
            },
            "textLabel": []
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "invyer8il": [
              "実施機関名",
              "電話番号"
            ]
          },
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        }
      },
      "layerBlending": "additive",
      "splitMaps": []
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 43.59401984322031,
      "longitude": 141.14463375422739,
      "pitch": 0,
      "zoom": 6.201792993981295,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [ ]:
# ウィジェットを使って notebook 内に地図を表示する
from keplergl import KeplerGl 
kepler = KeplerGl(height=400, config=config)
kepler.add_data(data=df_result, name="rubella")

kepler

In [ ]:
# HTMLファイルに出力する
kepler.save_to_html(file_name="docs/rubella_antibody_test.html")

可視化の結果を[ブラウザで表示](./docs/rubella_antibody_test.html)する